# Installation et imports des librairies

**Installation**

In [1]:
#Environnement Gym avec chargement des jeux Atari
!pip install gym[atari]
#Visualisation du jeu dans Google Colab (création d'une vidéo)
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

  Created wheel for colabgymrender: filename=colabgymrender-1.0.8-cp37-none-any.whl size=2511 sha256=08377aa43cf43b53b59a72fe48bff3d791b84c02c79e47533427861f5f220c1d
  Stored in directory: /root/.cache/pip/wheels/8a/d1/e8/ef1d4f6e536cc6b965b28c859ccd2a7f7ab123b94ef4007712
Successfully built colabgymrender


**Imports**

In [2]:
import gym
import random
from colabgymrender.recorder import Recorder

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2686976/45929032 bytes (5.9%)6176768/45929032 bytes (13.4%)9789440/45929032 bytes (21.3%)13410304/45929032 bytes (29.2%)17039360/45929032 bytes (37.1%)20430848/45929032 bytes (44.5%)23592960/45929032 bytes (51.4%)27123712/45929032 bytes (59.1%)30621696/45929032 bytes (66.7%)33849344/45929032 bytes (73.7%)37109760/45929032 bytes (80.8%)40632320/45929032 bytes (88.5%)

# Environnement Gym

**Création de l'environnement Pong**

Roms Atari disponible sur https://gym.openai.com/envs/#atari

In [ ]:
#Environnement (Roms disponiblent sur )
env = gym.make('Pong-v0')

**Récupérations d'informations utiles sur l'environnement**

In [ ]:
#Récupération des éléments de l'environnment pour le réseau de neuronne
#Hauteur/Largeur des images, channels => aucune documentation trouvé sur le sujet
height, width, channels = env.observation_space.shape

#Nombre d'actions disponibles pour le jeu chargé
actions = env.action_space.n
print("Nombre d'actions qu'un joueur peut choisir sur Pong :",actions)

#Les actions disponibles pour le jeu chargé
print("Les actions disponibles pour un joueur sont :")
env.unwrapped.get_action_meanings()



Nombre d'actions qu'un joueur peut choisir sur Pong : 6
Les actions disponibles pour un joueur sont :


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

**Un agent jouant à Pong avec des actions aléatoires** 

In [ ]:
#Lancement de l'enregistrement du jeu
env = Recorder(env,'./video')

#@markdown ##Paramètre modifiable :

#Nombre de partie
nombrePartie =  2#@param {type:"number"}

#Pour chaque partie
for unePartie in range(1, nombrePartie + 1):
  state = env.reset()
  done = False
  score = 0

  #=======================================================
  #@markdown ##Variables :
  #@markdown * **observation (objet) :** Objet spécifique à l'environnement comme les données d'images par exemple).
  #@markdown * **reward (float) :** Récompense obtenu par l'agent qui joue contre le jeu. 
  #@markdown * **done (boolean) :** Si le jeu est terminé ou non.
  #@markdown * **info (dictionnaire) :** Autres informations diverses fourni par l'environnement.
  #=======================================================

  #Tant que la partie n'est pas terminé
  while not done:
    action = random.choice([0,1,2,3,4,5])
    observation, reward, done, info = env.step(action)
    score += reward

  #Affichage console de la partie et des gains de l'agent jouant contre le jeu.
  print('\nPartie : {} Score : {}'.format(unePartie,score))

  #Affichage de la partie sous forme de vidéo
  env.play()
env.close()

 12%|█▏        | 137/1162 [00:00<00:00, 1320.32it/s]


Partie : 1 Score : -20.0


100%|█████████▉| 1161/1162 [00:00<00:00, 1513.85it/s]


 12%|█▏        | 127/1084 [00:00<00:00, 1269.24it/s]


Partie : 2 Score : -21.0


100%|█████████▉| 1083/1084 [00:00<00:00, 1467.73it/s]


# Notre réseau de neuronnes à convolution

**Installation des librairies**

In [ ]:
!pip install keras-rl2
#Environnement Gym
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]
#Visualisation du jeu dans Google Colab
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

**Imports**

In [3]:
#! python3.8
import gym
#import retro
import numpy as np
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from colabgymrender.recorder import Recorder

In [ ]:
#@markdown ##Paramètres
#Définition de l'environnement Gym
ENV_NAME = 'Pong-v0' #@param {type:"string"}
NOMBRE_APPRENTISSAGES =  20000#@param {type:"number"} 
NOMBRE_TEST =  20#@param {type:"number"}
env = gym.make(ENV_NAME)

#Définition d'une fonction qui créée un modèle.
def getModel(env):
  model = Sequential()
  model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
  model.add(Dense(16))
  model.add(Activation('relu'))
  model.add(Dense(16))
  model.add(Activation('relu'))
  model.add(Dense(16))
  model.add(Activation('relu'))
  model.add(Dense(env.action_space.n)) #Nombre d'actions disponibles
  model.add(Activation('linear'))
  return model

#Définition d'une fonction qui créée un agent.
def getAgent(env,model):
  memory = SequentialMemory(limit=500, window_length=1)
  policy = BoltzmannQPolicy()
  dqn = DQNAgent(model=model, nb_actions = env.action_space.n, memory = memory,
               nb_steps_warmup=10, target_model_update=1e-2, policy=policy)
  return dqn
  
model = getModel(env)
agent = getAgent(env,model)

#Entrainement
agent.compile(Adam(lr=1e-3), metrics=['mae'])
agent.fit(env,nb_steps=NOMBRE_APPRENTISSAGES, visualize=False, verbose=2)

#Sauvegarde des poids obtenue lors de l'apprentissage
agent.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True )

#Chargement des poids sauvegardés
agent.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))
env = Recorder(env,'./video')
#Tests de l'apprentissage
scores = agent.test(env,nb_episodes=NOMBRE_TEST,visualize=True )
#env.play()
print(np.mean(scores.history['episode_reward']))

env.close()